# Age Contour Disclosure Manager Code

## What this notebook does


This notebook has been set up to read in csvs, jpegs, and html files which have been sent to the review bucket, so that they can be checked for quality and disclosure. All items which are requested for export first needs to be Disclosure controlled by a seperate DisCO (Disclosure Control Officer), this cannot be the same person who ran the data and requested the export. Once disclosure controlled a Data Journey manager can be notified to move the data ready for export.

## Setup

Authentification for GCP and libraries

In [ ]:
options("googleAuthR.httr_oauth_cache" = "gce.oauth")
googleAuthR::gar_gce_auth()
devtools::install("../gcptools", upgrade = FALSE)
library(googleCloudStorageR)
library(readr)

This ensures that people cannot commit notebooks containing evalutated outputs.

In [ ]:
gcptools::commit_hooks_setup("/home/jupyter/CIS_age_contour") #[IMPORTANT SECURITY FEATURE DO NOT REMOVE] 

custom parse function to suppress warning, because default gives a warning about missing column name.

In [ ]:
f <- function(object){
  suppressWarnings(httr::content(object, encoding = "UTF-8"))
}

## Get csv files from the review bucket

<FONT COLOR="RED"> **INSTRUCTION:**</FONT> Select the csv file you want to review by replacing the string below with a filepath to the csv

In [ ]:
object_name <- "20221121_agecontour_swabs_OneDayMissing/probs_age_contour_swabs_20221115_DTS221212_1359UTC.csv" #or "<yyyymmdd>_agecontour_antibodies/<filename>.csv"

In [ ]:
data = googleCloudStorageR::gcs_get_object(object_name,
                                           bucket = "polestar-prod-review",
                                           parseFunction = f)

<FONT COLOR="RED"> **INSTRUCTION:**</FONT> Check the contents of your csv by using the head, tail, or summary functions

In [ ]:
tail(data, 20) #or head(data), summary(data) etc

## Get jpeg files from the review bucket:

<FONT COLOR="RED"> **INSTRUCTION:**</FONT> Set folderapth to your chosen folder, in the format below.

In [ ]:
folderpath <- "20221114_agecontour_swabs"

Create pattern to match on (see link for more info on pattern matching. https://evoldyn.gitlab.io/evomics-2018/ref-sheets/R_strings.pdf)

In [ ]:
pattern_to_match = paste0("^", 
                          folderpath,
                          "/.*", #this matches all characters
                          ".jpeg")
print(paste0("This is the pattern that is being matched on:    ", pattern_to_match))

Download all files which match this pattern from the review bucket

In [ ]:
jpegs <- grep(x = googleCloudStorageR::gcs_list_objects("polestar-prod-review")$name,
               pattern = pattern_to_match,
               value = TRUE)
jpegs

Pull all of the jpeg images into a single list for plotting

In [ ]:
jpeg_list <- lapply(X = jpegs,
                    FUN = googleCloudStorageR::gcs_get_object,
                    bucket = "polestar-prod-review")

Plot jpegs, assuming square images of 3000px resolution 

In [ ]:
for(fig in 1:length(jpeg_list)){
    plot(0:3000, type = 'n')
 rasterImage(image = jpeg_list[[fig]], 0, 0, 3000, 3000)
} 

## Get QA reports from the review bucket:

If 'docs' folder does not exist in the local directory, then create it. Either way, set the working directory to the docs folder to save outputs

In [ ]:
main_directory <- "/home/jupyter/"
sub_directory <- "QA_reports"

if (file.exists(sub_directory)){
    setwd(file.path(main_directory, sub_directory))
} else {
    dir.create(file.path(main_directory, sub_directory))
    setwd(file.path(main_directory, sub_directory))   
}

<FONT COLOR="RED"> **INSTRUCTION:**</FONT> Replace the string in the 'file' argument to select the correct file from the review bucket to download files into your local QA_Report folder

In [ ]:
gcptools::download_qa_report_to_notebook(file = "20221121_agecontour_swabs_OneDayMissing/qareport_age_contour_swabs_20221115_DTS221212_1359UTC.html", #or "<yyyymmdd>_agecontour_antibodies/<filename>.html"
                                         bucket = "polestar-prod-review") 

<FONT COLOR="RED"> **INSTRUCTION:**</FONT> Locate the QA_reports folder at the repo level of folders (i.e. where you would go to navigate to other repo's) to review the report